## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-13-01-11-45 +0000,nypost,Cincinnati leaders ‘demanding prosecution’ of ...,https://nypost.com/2025/08/12/us-news/cincinna...
1,2025-08-13-01-00-44 +0000,nyt,Fact-Checking Trump’s False and Misleading Cla...,https://www.nytimes.com/2025/08/12/us/politics...
2,2025-08-13-01-00-00 +0000,wsj,Ukraine’s Once Nimble Army Is Mired in Soviet ...,https://www.wsj.com/world/ukraine-russia-army-...
3,2025-08-13-01-00-00 +0000,wsj,The Recipe Behind the Trump Family’s Crypto Ri...,https://www.wsj.com/finance/currencies/trump-f...
4,2025-08-13-00-59-46 +0000,nyt,Trump’s D.C. Crackdown Plan Begins to Take Sha...,https://www.nytimes.com/2025/08/12/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2409/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,56
490,china,12
48,new,12
41,report,11
183,will,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
291,2025-08-12-01-47-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...,117
230,2025-08-12-10-35-02 +0000,nypost,JD Vance says Elon Musk should return to White...,https://nypost.com/2025/08/12/us-news/jd-vance...,101
117,2025-08-12-19-26-00 +0000,wsj,President Trump on Tuesday appeared to call fo...,https://www.wsj.com/economy/trump-calls-on-gol...,99
85,2025-08-12-21-08-00 +0000,wsj,The Trump administration is considering change...,https://www.wsj.com/politics/policy/trump-jobs...,97
225,2025-08-12-11-01-50 +0000,bbc,Trump deploys National Guard to Washington DC ...,https://www.bbc.com/news/articles/cm2110me5g4o...,96


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
291,117,2025-08-12-01-47-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...
119,59,2025-08-12-19-21-00 +0000,wsj,The State Department released a long-delayed h...,https://www.wsj.com/politics/policy/state-depa...
54,58,2025-08-12-22-22-09 +0000,nypost,NYC ex-cons slice and dice their way through n...,https://nypost.com/2025/08/12/us-news/nyc-ex-c...
151,56,2025-08-12-17-33-00 +0000,wsj,Crypto Entrepreneur Do Kwon Pleads Guilty to F...,https://www.wsj.com/finance/currencies/crypto-...
22,49,2025-08-12-23-51-56 +0000,nyt,White House Will ‘Definitely’ Host a U.F.C. Fi...,https://www.nytimes.com/2025/08/12/us/politics...
57,41,2025-08-12-22-15-48 +0000,cbc,Video appears to show moment Palestinian activ...,https://www.cbc.ca/news/world/awdah-hathaleen-...
164,38,2025-08-12-16-42-05 +0000,bbc,Israel bombards Gaza City as UK and allies dem...,https://www.bbc.com/news/articles/clyj0dd0qj9o...
215,35,2025-08-12-12-03-12 +0000,nyt,Gaming Out the Price of a Trump Trade Deal Wit...,https://www.nytimes.com/2025/08/12/business/de...
292,34,2025-08-12-01-29-17 +0000,bbc,Two dead and 10 injured in Pennsylvania steel ...,https://www.bbc.com/news/articles/ckgjj49neldo...
62,34,2025-08-12-22-07-14 +0000,nypost,"2 teen boys shot, wounded when gunman wearing ...",https://nypost.com/2025/08/12/us-news/2-teen-b...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
